In [163]:
import pandas as pd

In [164]:
def calc_indent(numb: int) -> str:
    if numb < 2:
        return ""
    binary = bin(numb)[3:]
    ret = map(lambda x: "│  " if x == "1" else "   ", binary[:-1])
    return "".join(ret) + ( "└──" if binary[-1] == "0" else "├──")


class Category:
    def __init__(self, name, image):
        self.id = None
        self.name = name
        self.children = []
        self.image = image
        
    def add_child(self, child):
        self.children.append(child)
    
    def add_child(self, name, image):
        self.children.append(Category(name, image))
        
    def add_children(self, children):
        self.children.extend(map(lambda x: Category(x[0],x[1]) ,children))
        
    def __str__(self, indent=1):
        ret = calc_indent(indent) + self.__repr__() + "\n"
        for child in self.children[:-1]:
            ret += child.__str__(indent*2+1)
        if len(self.children) > 0:
            ret += self.children[-1].__str__(indent*2)
        return ret
    
    
    def __repr__(self):
        return f'[{self.id if self.id else "?"}] {self.name} [{self.image if self.image else "None"}]'


    def get_child(self, name):
        for child in self.children:
            if child.name == name:
                return child
        return None

    def __getitem__(self, key):
        if hasattr(self, 'children') and isinstance(self.children, (list, dict)):
            return self.get_child(key)
        else:
            raise AttributeError("The 'children' attribute does not exist or is not subscriptable.")

In [165]:
Men = Category('Men', None)
Men.add_children([
    ('Clothing', None),
    ('Suits', None),
    ('Shoes', None),
])

Men['Clothing'].add_children([
    ('Coats', None),
    ('Dresses', None),
    ('Jumpsuits', None),
    ('Jackets', None),
    ('Suit Jackets', None),
    ('Sweaters', None),
    ('Cardigans', None),
    ('Shirts', None),
    ('Leather', None),
    ('T-Shirts', None),
    ('Tops', None),
    ('Sweatshirts', None),
    ('Trousers', None),
    ('Shirts', None),
    ('Jeans', None),
    ('Pikinis', None),
    ('Swimsuits', None),
])
Men['Suits'].add_children([
    ('Suits', None),
    ('Blazers', None),
    ('Waistcoats', None),
    ('Ties', None),
    ('Bow Ties', None),
    ('Pocket Squares', None),
    ('Cufflinks', None),
    ('Tie Clips', None),
])
print(Men)

[?] Men [None]
├──[?] Clothing [None]
│  ├──[?] Coats [None]
│  └──[?] Dresses [None]
├──[?] Suits [None]
└──[?] Shoes [None]

